### 将标签转换成索引
优点：便于计算和索引，使用索引更高效

In [1]:
# 设置 tag2id, id2tag
tag2id, id2tag = {}, {} # map tag to id. tag2id:{"VB":0, "NNP":1 ...}, id2tag:{0:"VB",1:"NNP"}

word2id, id2word = {}, {}


for line in open('data/traindata.txt'):
    items = line.split('/')
    word, tag = items[0], items[1].rstrip()  # 抽取每一行的单词和词性

    # 构建 word2id
    if word not in word2id:
        word2id[word] = len(word2id)
        id2word[len(id2word)] = word
    if tag not in tag2id:
        tag2id[tag] = len(tag2id)
        id2tag[len(id2tag)] = tag
    

M = len(word2id)  # M: 词典的大小 
N = len(tag2id)   # N: 词性的种类

In [2]:
print(M,N)
print(tag2id)

18978 54
{'NNP': 0, ',': 1, 'VBG': 2, 'TO': 3, 'VB': 4, 'NN': 5, 'IN': 6, 'JJ': 7, 'VBD': 8, 'NNS': 9, 'CD': 10, 'CC': 11, 'PRP': 12, 'MD': 13, 'DT': 14, '.': 15, 'VBZ': 16, 'VBN': 17, 'WDT': 18, 'VBP': 19, 'POS': 20, 'RB': 21, '$': 22, 'PRP$': 23, ':': 24, 'JJR': 25, '``': 26, "''": 27, 'WP': 28, 'JJS': 29, 'WRB': 30, 'RBR': 31, 'NNPS': 32, 'RP': 33, 'WP$': 34, 'EX': 35, '(': 36, ')': 37, 'PDT': 38, 'RBS': 39, 'FW': 40, 'UH': 41, 'SYM': 42, 'LS': 43, '#': 44, 'VBG|NN': 45, 'JJ|NN': 46, 'RB|IN': 47, 'NNS|NN': 48, 'VBN|JJ': 49, 'VB|NN': 50, 'RBR|JJR': 51, 'NN|NNS': 52, 'JJ|RB': 53}


### 构建和填充 pi，A，b

In [3]:
import numpy as np

# 初始化
pi = np.zeros(N)  # pi 表示每个词性出现在句首的概率是多少 N
A = np.zeros((N, M)) # A[i][j]：给定tag i，出现单词j 的概率
B = np.zeros((N, N)) # B[i][j]: 先前状态（词性）是i，之后转换成状态（词性）j的概率

In [4]:
# 填充 pi，A，B
prev_tag = ""
for line in open('data/traindata.txt'):
    items = line.split('/')
    # 得到当前单词和词性的索引
    wordId, tagId = word2id[items[0]], tag2id[items[1].rstrip()]

    if prev_tag == "": # 这意味着句子的开始
        pi[tagId] += 1 # 先统计句首的次数
        A[tagId][wordId] += 1 # 某词性下单词出现的概率（不管是不是句首）
    else:              # 如果不是句首
        A[tagId][wordId] +=1
        B[tag2id[prev_tag]][tagId] += 1  # 统计之前词性下，当前词性的次数
    
    if items[0] == ".": 
        prev_tag = ""
    else:
        prev_tag = items[1].rstrip()


# normalize 转换成概率
pi = pi/sum(pi)
for i in range(N):
    A[i] /= sum(A[i])
    B[i] /= sum(B[i])


# 到此为止，计算完模型的所有参数：pi，A，B

In [5]:
print(pi) # 看那些词性出现在句首的概率

[1.81324111e-01 0.00000000e+00 1.00049407e-02 3.33498024e-03
 3.95256917e-03 3.68083004e-02 1.11660079e-01 3.66847826e-02
 6.17588933e-04 3.81669960e-02 8.76976285e-03 5.18774704e-02
 6.02766798e-02 2.47035573e-04 2.17267787e-01 0.00000000e+00
 1.48221344e-03 6.05237154e-03 8.64624506e-04 2.47035573e-04
 0.00000000e+00 4.73073123e-02 0.00000000e+00 7.16403162e-03
 1.72924901e-03 2.09980237e-03 7.53458498e-02 6.36116601e-02
 2.59387352e-03 1.85276680e-03 5.92885375e-03 1.97628458e-03
 2.84090909e-03 0.00000000e+00 0.00000000e+00 2.71739130e-03
 5.92885375e-03 5.92885375e-03 9.88142292e-04 3.70553360e-04
 1.23517787e-04 0.00000000e+00 0.00000000e+00 1.85276680e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [6]:
print(A[0]) # 统计出给定'NNP'词性，每个单词出现的情况
print(word2id) 
print(tag2id)

[5.16155673e-04 0.00000000e+00 0.00000000e+00 ... 5.16155673e-05
 0.00000000e+00 0.00000000e+00]
{'Newsweek': 0, ',': 1, 'trying': 2, 'to': 3, 'keep': 4, 'pace': 5, 'with': 6, 'rival': 7, 'Time': 8, 'magazine': 9, 'announced': 10, 'new': 11, 'advertising': 12, 'rates': 13, 'for': 14, '1990': 15, 'and': 16, 'said': 17, 'it': 18, 'will': 19, 'introduce': 20, 'a': 21, 'incentive': 22, 'plan': 23, 'advertisers': 24, '.': 25, 'The': 26, 'ad': 27, 'from': 28, 'unit': 29, 'of': 30, 'the': 31, 'Washington': 32, 'Post': 33, 'Co.': 34, 'is': 35, 'second': 36, 'has': 37, 'offered': 38, 'in': 39, 'three': 40, 'years': 41, 'Plans': 42, 'that': 43, 'give': 44, 'discounts': 45, 'maintaining': 46, 'or': 47, 'increasing': 48, 'spending': 49, 'have': 50, 'become': 51, 'permanent': 52, 'fixtures': 53, 'at': 54, 'news': 55, 'weeklies': 56, 'underscore': 57, 'fierce': 58, 'competition': 59, 'between': 60, 'Warner': 61, 'Inc.': 62, "'s": 63, 'Mortimer': 64, 'B.': 65, 'Zuckerman': 66, 'U.S.': 67, 'News': 68,

### 找最优的词性序列-维特比算法

In [7]:
# 处理平滑方法
def log(v):
    if v == 0:
        return np.log(v+0.00001)
    return np.log(v)

In [8]:
import math
# 维特比算法
def viterbi(x, pi, A, B):
    """
    x：user input string/sentence  x: "l like playing soccer"
    pi：intial prbalilty of tags
    A：给定tag，每个单词出现的概率
    B：tag间的转移概率
    """

    x = [word2id[word] for word in x.split(" ")] # x:[231,23123,313...]
    T = len(x)

    dp = np.zeros((T,N))  # dp[i][j]: w1...wi，假设wi的tag是第j个tag
    ptr = np.array([[0 for x in range(N)] for y in range(T)])        # 存储最有路径从哪过来的 T*N

    # 初始化第一列，就是 logP(w1|z1) + logP(z1)
    for j in range(N):
        dp[0][j] = log(pi[j]) + log(A[j][x[0]])
    

    for i in range(1,T): # 每个单词
        for j in range(N): # 每个词性
            dp[i][j] = -999999
            for k in range(N):  # 从每一个k达到j (所有列中取最大的值，付给dp[i][j])
                score = dp[i-1][k] + log(B[k][j]) + log(A[j][x[i]])
                if score > dp[i][j]:
                    dp[i][j] = score
                    ptr[i][j] = k

    
    # 把最好的 tag sequence 打印出来
    best_seq = [0]*T   # base_seq = [2,5,25,4...]
    # 1.找出对应于最后一个单词的词性
    best_seq[T-1] = np.argmax(dp[T-1]) 

    # 2.通过从后到前的循环，依次求出每个单词的词性
    for i in range(T-2, -1, -1):
        best_seq[i] = ptr[i+1][best_seq[i+1]]
    

    # 到目前为止，base_seq 存放对应x的词性序列
    for i in range(len(best_seq)):
        print(id2tag[best_seq[i]])

In [9]:
x = "Social Security number , passport number and details about the service provided"
viterbi(x, pi, A, B)

NNP
NNP
NN
,
DT
NN
CC
NNS
IN
DT
NN
VBD
